In [2]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation
from sklearn.cluster import KMeans

!pip install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

! pip install folium==0.5.0
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
     |████████████████████████████████| 79 kB 6.8 MB/s  eta 0:00:01
  Created wheel for folium: file

In [3]:
# Create Eindhoven dataframe
csv_url_eind = 'https://data.eindhoven.nl/explore/dataset/postcode-buurt-wijk/download/?format=csv&timezone=Europe/Berlin&lang=en&use_labels_for_header=true&csv_separator=%3B'
df_eind = pd.read_csv(csv_url_eind, sep = ';', engine = 'python')

# Drop unneeded columns
df_eind.drop(['STRAATNAAM', 'POSTCODE6','HSNR_laag', 'HSNR_hoog', 'HSNR_aant', 'BUURTCODE', 'WIJKCODE', 'WIJKNAAM','x_rd','y_rd'], axis = 1, inplace = True)


#Rename column
df_eind.columns = ['Postcode', 'Neighborhood', 'District Code', 'District Name', 'Latitude', 'Longitude']

#Drop duplicate values due to long streets in the dataset
df_eind=df_eind.drop_duplicates(subset='Postcode')
df_eind.head()


,Postcode,Neighborhood,District Code,District Name,Latitude,Longitude
0,5612.0,Limbeek-Noord,4,Stadsdeel Woensel-Zuid,51.449979,5.462990
1,5611.0,Binnenstad,1,Stadsdeel Centrum,51.437246,5.481639
2,5641.0,'t Hofke,3,Stadsdeel Tongelre,51.448200,5.519191
6,5621.0,Woensel-West,4,Stadsdeel Woensel-Zuid,51.453603,5.459020
7,5622.0,Mensfort,4,Stadsdeel Woensel-Zuid,51.464195,5.461028


In [4]:
df_eind['City']='Eindhoven'
df_eind.head()

,Postcode,Neighborhood,District Code,District Name,Latitude,Longitude,City
0,5612.0,Limbeek-Noord,4,Stadsdeel Woensel-Zuid,51.449979,5.462990,Eindhoven
1,5611.0,Binnenstad,1,Stadsdeel Centrum,51.437246,5.481639,Eindhoven
2,5641.0,'t Hofke,3,Stadsdeel Tongelre,51.448200,5.519191,Eindhoven
6,5621.0,Woensel-West,4,Stadsdeel Woensel-Zuid,51.453603,5.459020,Eindhoven
7,5622.0,Mensfort,4,Stadsdeel Woensel-Zuid,51.464195,5.461028,Eindhoven


In [5]:
# Amount of unique categories 
print('There are {} uniques categories.'.format(len(df_eind['Postcode'].unique())))

There are 34 uniques categories.


In [6]:
CLIENT_ID = '45O4XNUMTSYU5O4V4I4NEE0BMRJ122KTZJODVO4SYAQC0KQR' # your Foursquare ID
CLIENT_SECRET = 'WMRCAPPWYAOKTM51TK3R3YNXKLO4LUO2D5FSOYVNU0WWZ3B0' # your Foursquare Secret
ACCESS_TOKEN = '024WRMUUBLQZ1Z1QBPFTVPTC2D3EVWROWYEPPOUAXVRIEELU' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 150
radius= 2000
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 45O4XNUMTSYU5O4V4I4NEE0BMRJ122KTZJODVO4SYAQC0KQR
CLIENT_SECRET:WMRCAPPWYAOKTM51TK3R3YNXKLO4LUO2D5FSOYVNU0WWZ3B0


In [7]:
address="Eindhoven, Noord-Brabant, the Netherlands"

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

51.4392648 5.478633


In [8]:
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '60dc884c9d13f207b7df8c14'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Binnenstad',
  'headerFullLocation': 'Binnenstad, Eindhoven',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 149,
  'suggestedBounds': {'ne': {'lat': 51.45726481800001,
    'lng': 5.507455630090952},
   'sw': {'lat': 51.42126478199998, 'lng': 5.4498103699090485}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5a02ed9ca6fe4d6ddd98d30a',
       'name': 'Yoghurt Barn Eindhoven',
       'location': {'address': 'Nieuwstraat 30',
        'lat': 51.44018675762301,
        'lng': 5.478956245148944,
        'labeledLatLngs': [{'label': 'display',
          'la

In [9]:
items = results['response']['groups'][0]['items']
items[0]

{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '5a02ed9ca6fe4d6ddd98d30a',
  'name': 'Yoghurt Barn Eindhoven',
  'location': {'address': 'Nieuwstraat 30',
   'lat': 51.44018675762301,
   'lng': 5.478956245148944,
   'labeledLatLngs': [{'label': 'display',
     'lat': 51.44018675762301,
     'lng': 5.478956245148944}],
   'distance': 105,
   'postalCode': '5611 DB',
   'cc': 'NL',
   'city': 'Eindhoven',
   'state': 'Noord-Brabant',
   'country': 'Nederland',
   'formattedAddress': ['Nieuwstraat 30', '5611 DB Eindhoven', 'Nederland']},
  'categories': [{'id': '512e7cae91d4cbb4e5efe0af',
    'name': 'Frozen Yogurt Shop',
    'pluralName': 'Frozen Yogurt Shops',
    'shortName': 'Yogurt',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/frozenyogurt_',
     'suffix': '.png'},
    'primary': True}],
  'photos': {'count': 0, 'groups': []}},
 'referralId': 'e-0

In [10]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [11]:
dataframe = json_normalize(items) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# filter the category for each row
dataframe_filtered['venue.categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean columns
dataframe_filtered.columns = [col.split('.')[-1] for col in dataframe_filtered.columns]

dataframe_filtered.drop(['address', 'distance','labeledLatLngs','country', 'state', 'formattedAddress', 'neighborhood', 'crossStreet','id','cc'], axis = 1, inplace = True)
dataframe_filtered.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  if __name__ == '__main__':


,name,categories,lat,lng,postalCode,city
0,Yoghurt Barn Eindhoven,Frozen Yogurt Shop,51.440187,5.478956,5611 DB,Eindhoven
1,De Bierbrigadier,Beer Store,51.436422,5.476816,NaN,Eindhoven
2,De Vooruitgang,Restaurant,51.439171,5.478755,5611 EB,Eindhoven
3,De Burger,Burger Joint,51.436799,5.478435,5611 GH,Eindhoven
4,DENF Coffee,Coffee Shop,51.439165,5.474124,NaN,Eindhoven


In [12]:
# Amount of unique categories 
print('There are {} uniques categories.'.format(len(dataframe_filtered['categories'].unique())))

There are 64 uniques categories.


In [13]:
len(dataframe_filtered)

100

In [14]:
# Most frequent categories
dataframe_filtered.groupby('categories').count().sort_values('name', ascending = False)

,name,lat,lng,postalCode,city
categories,,,,,
Restaurant,6,6,6,6,6
French Restaurant,5,5,5,5,5
Bar,4,4,4,3,3
Supermarket,4,4,4,4,4
Coffee Shop,4,4,4,3,4
...,...,...,...,...,...
Food Court,1,1,1,1,1
Frozen Yogurt Shop,1,1,1,1,1
Gay Bar,1,1,1,1,1


In [15]:
dataframe_filtered['Postcode']=dataframe_filtered['postalCode'].str[0:4]
dataframe_filtered

,name,categories,lat,lng,postalCode,city,Postcode
0,Yoghurt Barn Eindhoven,Frozen Yogurt Shop,51.440187,5.478956,5611 DB,Eindhoven,5611
1,De Bierbrigadier,Beer Store,51.436422,5.476816,NaN,Eindhoven,NaN
2,De Vooruitgang,Restaurant,51.439171,5.478755,5611 EB,Eindhoven,5611
3,De Burger,Burger Joint,51.436799,5.478435,5611 GH,Eindhoven,5611
4,DENF Coffee,Coffee Shop,51.439165,5.474124,NaN,Eindhoven,NaN
...,...,...,...,...,...,...,...
95,Mei Wah,Chinese Restaurant,51.425899,5.489555,5615AB,Eindhoven,5615
96,Albert Heijn,Supermarket,51.440534,5.453364,5616 GM,Eindhoven,5616
97,Feel Good Market,Flea Market,51.447724,5.456817,NaN,Eindhoven,NaN
98,Kruidvat,Drugstore,51.441137,5.453247,5611,Eindhoven,5611


In [16]:
# One hot encoding
dfeind_venues_onehot = pd.get_dummies(dataframe_filtered[['categories']], prefix = "", prefix_sep = "")

# Add Zipcode column back to dataframe
col1 = dataframe_filtered['city']
col2 = dataframe_filtered['Postcode'] 
 
# Move Zipcode column to the first column
dfeind_venues_onehot.insert(0, 'Postcode', col2)
dfeind_venues_onehot.insert(0, 'city', col1)

dfeind_venues_onehot

,city,Postcode,Afghan Restaurant,Asian Restaurant,Bagel Shop,Bar,Bed & Breakfast,Beer Store,Bookstore,Breakfast Spot,...,Pub,Restaurant,Rock Club,Skate Park,Sporting Goods Shop,Supermarket,Tapas Restaurant,Thai Restaurant,Theater,Vegetarian / Vegan Restaurant
0,Eindhoven,5611,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Eindhoven,NaN,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Eindhoven,5611,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,Eindhoven,5611,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Eindhoven,NaN,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Eindhoven,5615,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
96,Eindhoven,5616,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
97,Eindhoven,NaN,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
98,Eindhoven,5611,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
dfeind_venues_onehot.dropna(inplace=True)
print('There are {} uniques categories.'.format(len(dfeind_venues_onehot['Postcode'].unique())))

There are 10 uniques categories.


In [18]:
dfeind_venues_onehot

,city,Postcode,Afghan Restaurant,Asian Restaurant,Bagel Shop,Bar,Bed & Breakfast,Beer Store,Bookstore,Breakfast Spot,...,Pub,Restaurant,Rock Club,Skate Park,Sporting Goods Shop,Supermarket,Tapas Restaurant,Thai Restaurant,Theater,Vegetarian / Vegan Restaurant
0,Eindhoven,5611,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Eindhoven,5611,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,Eindhoven,5611,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,Eindhoven,5611,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,Eindhoven,5611,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,Eindhoven,5615,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
95,Eindhoven,5615,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
96,Eindhoven,5616,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
98,Eindhoven,5611,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
dfeind_venues_onehot.groupby('Postcode').count().sort_values('Postcode', ascending = False)

,city,Afghan Restaurant,Asian Restaurant,Bagel Shop,Bar,Bed & Breakfast,Beer Store,Bookstore,Breakfast Spot,Brewery,...,Pub,Restaurant,Rock Club,Skate Park,Sporting Goods Shop,Supermarket,Tapas Restaurant,Thai Restaurant,Theater,Vegetarian / Vegan Restaurant
Postcode,,,,,,,,,,,,,,,,,,,,,
5654,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
5652,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
5644,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
5617,7,7,7,7,7,7,7,7,7,7,...,7,7,7,7,7,7,7,7,7,7
5616,8,8,8,8,8,8,8,8,8,8,...,8,8,8,8,8,8,8,8,8,8
5615,8,8,8,8,8,8,8,8,8,8,...,8,8,8,8,8,8,8,8,8,8
5614,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
5613,4,4,4,4,4,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4
5612,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2


In [20]:
eind_grouped = dfeind_venues_onehot.groupby('Postcode').mean().reset_index()
eind_grouped.head()

,Postcode,Afghan Restaurant,Asian Restaurant,Bagel Shop,Bar,Bed & Breakfast,Beer Store,Bookstore,Breakfast Spot,Brewery,...,Pub,Restaurant,Rock Club,Skate Park,Sporting Goods Shop,Supermarket,Tapas Restaurant,Thai Restaurant,Theater,Vegetarian / Vegan Restaurant
0,5611,0.0,0.000,0.041667,0.041667,0.020833,0.0,0.020833,0.041667,0.041667,...,0.041667,0.0625,0.0,0.0,0.0,0.000,0.0,0.0,0.000,0.020833
1,5612,0.0,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.0000,0.0,0.0,0.0,0.000,0.0,0.0,0.000,0.000000
2,5613,0.0,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.2500,0.0,0.0,0.0,0.250,0.0,0.0,0.000,0.000000
3,5614,0.0,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.0000,0.0,0.0,0.0,0.000,0.0,0.0,0.000,0.000000
4,5615,0.0,0.125,0.000000,0.125000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.0000,0.0,0.0,0.0,0.125,0.0,0.0,0.125,0.000000


In [21]:
num_top_venues = 5

for hood in eind_grouped['Postcode']:
    print("----"+hood+"----")
    temp = eind_grouped[eind_grouped['Postcode'] == hood].T.reset_index() #We transpose the information
    temp.columns = ['venue','freq'] #We add title to the serie
    temp = temp.iloc[1:] #We delete first row which contain Neighborhood name
    temp['freq'] = temp['freq'].astype(float) #We convert number to float
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----5611----
               venue  freq
0  French Restaurant  0.06
1         Restaurant  0.06
2         Bagel Shop  0.04
3                Bar  0.04
4                Pub  0.04


----5612----
                venue  freq
0         College Gym   0.5
1      Cigkofte Place   0.5
2  Italian Restaurant   0.0
3              Museum   0.0
4           Multiplex   0.0


----5613----
               venue  freq
0               Café  0.25
1     History Museum  0.25
2        Supermarket  0.25
3         Restaurant  0.25
4  Afghan Restaurant  0.00


----5614----
               venue  freq
0      Movie Theater   1.0
1  Afghan Restaurant   0.0
2   Asian Restaurant   0.0
3      Grocery Store   0.0
4     History Museum   0.0


----5615----
                venue  freq
0  Chinese Restaurant  0.12
1             Theater  0.12
2                 Bar  0.12
3    Asian Restaurant  0.12
4         Supermarket  0.12


----5616----
                 venue  freq
0           Restaurant  0.12
1  Sporting Goods Shop  0.12
2  

In [22]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [23]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Postcode']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
postcode_venues_sorted = pd.DataFrame(columns=columns)
postcode_venues_sorted['Postcode'] = eind_grouped['Postcode']

for ind in np.arange(eind_grouped.shape[0]):
    postcode_venues_sorted.iloc[ind, 1:] = return_most_common_venues(eind_grouped.iloc[ind, :], num_top_venues)

postcode_venues_sorted.head(90)

,Postcode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,5611,Restaurant,French Restaurant,Brewery,Hotel,Café,Pub,Multiplex,Coffee Shop,Breakfast Spot,Bar
1,5612,College Gym,Cigkofte Place,Vegetarian / Vegan Restaurant,Cocktail Bar,Food Court,Flea Market,Fish & Chips Shop,Fast Food Restaurant,Ethiopian Restaurant,Electronics Store
2,5613,Supermarket,History Museum,Café,Restaurant,Vegetarian / Vegan Restaurant,Department Store,Coffee Shop,College Gym,Concert Hall,Deli / Bodega
3,5614,Movie Theater,Vegetarian / Vegan Restaurant,French Restaurant,Flea Market,Fish & Chips Shop,Fast Food Restaurant,Ethiopian Restaurant,Electronics Store,Drugstore,Discount Store
4,5615,Theater,Asian Restaurant,Supermarket,Bar,French Restaurant,Fast Food Restaurant,Chinese Restaurant,Electronics Store,Discount Store,Concert Hall
5,5616,Movie Theater,Supermarket,Sporting Goods Shop,Electronics Store,Grocery Store,Restaurant,Climbing Gym,Coffee Shop,Concert Hall,Clothing Store
6,5617,Deli / Bodega,Ice Cream Shop,Office,Skate Park,Rock Club,Restaurant,Hotel Bar,Vegetarian / Vegan Restaurant,Coffee Shop,College Gym
7,5644,History Museum,Vegetarian / Vegan Restaurant,French Restaurant,Flea Market,Fish & Chips Shop,Fast Food Restaurant,Ethiopian Restaurant,Electronics Store,Drugstore,Discount Store
8,5652,Supermarket,Vegetarian / Vegan Restaurant,Clothing Store,Flea Market,Fish & Chips Shop,Fast Food Restaurant,Ethiopian Restaurant,Electronics Store,Drugstore,Discount Store
9,5654,French Restaurant,Asian Restaurant,Cocktail Bar,Food Court,Flea Market,Fish & Chips Shop,Fast Food Restaurant,Ethiopian Restaurant,Electronics Store,Drugstore


# Clustering the Postcodes

In [24]:
# set number of clusters
kclusters = 5

eind_grouped_clustering = eind_grouped.drop('Postcode', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(eind_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 0, 1, 2, 1, 1, 1, 4, 3, 1], dtype=int32)

In [25]:
labels = pd.DataFrame(kmeans.labels_, columns = ['Label'])
labels.head()

,Label
0,1
1,0
2,1
3,2
4,1


In [26]:
eind_labelled = labels.merge(postcode_venues_sorted, left_index=True, right_index=True)
eind_labelled["Postcode"] = eind_labelled.Postcode.astype(float)

In [29]:
eind_label = df_eind.merge(eind_labelled, left_on='Postcode', right_on='Postcode')
eind_label

,Postcode,Neighborhood,District Code,District Name,Latitude,Longitude,City,Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,5612.0,Limbeek-Noord,4,Stadsdeel Woensel-Zuid,51.449979,5.462990,Eindhoven,0,College Gym,Cigkofte Place,Vegetarian / Vegan Restaurant,Cocktail Bar,Food Court,Flea Market,Fish & Chips Shop,Fast Food Restaurant,Ethiopian Restaurant,Electronics Store
1,5611.0,Binnenstad,1,Stadsdeel Centrum,51.437246,5.481639,Eindhoven,1,Restaurant,French Restaurant,Brewery,Hotel,Café,Pub,Multiplex,Coffee Shop,Breakfast Spot,Bar
2,5614.0,Tuindorp,2,Stadsdeel Stratum,51.428464,5.492676,Eindhoven,2,Movie Theater,Vegetarian / Vegan Restaurant,French Restaurant,Flea Market,Fish & Chips Shop,Fast Food Restaurant,Ethiopian Restaurant,Electronics Store,Drugstore,Discount Store
3,5615.0,Bloemenplein,2,Stadsdeel Stratum,51.425888,5.486470,Eindhoven,1,Theater,Asian Restaurant,Supermarket,Bar,French Restaurant,Fast Food Restaurant,Chinese Restaurant,Electronics Store,Discount Store,Concert Hall
4,5644.0,Gerardusplein,2,Stadsdeel Stratum,51.422810,5.485955,Eindhoven,4,History Museum,Vegetarian / Vegan Restaurant,French Restaurant,Flea Market,Fish & Chips Shop,Fast Food Restaurant,Ethiopian Restaurant,Electronics Store,Drugstore,Discount Store
5,5616.0,"Eliasterrein, Vonderkwartier",6,Stadsdeel Strijp,51.437736,5.466534,Eindhoven,1,Movie Theater,Supermarket,Sporting Goods Shop,Electronics Store,Grocery Store,Restaurant,Climbing Gym,Coffee Shop,Concert Hall,Clothing Store
6,5652.0,Het Ven,6,Stadsdeel Strijp,51.441151,5.443418,Eindhoven,3,Supermarket,Vegetarian / Vegan Restaurant,Clothing Store,Flea Market,Fish & Chips Shop,Fast Food Restaurant,Ethiopian Restaurant,Electronics Store,Drugstore,Discount Store
7,5613.0,Lakerlopen,3,Stadsdeel Tongelre,51.438026,5.499375,Eindhoven,1,Supermarket,History Museum,Café,Restaurant,Vegetarian / Vegan Restaurant,Department Store,Coffee Shop,College Gym,Concert Hall,Deli / Bodega
8,5654.0,Blaarthem,7,Stadsdeel Gestel,51.424460,5.457675,Eindhoven,1,French Restaurant,Asian Restaurant,Cocktail Bar,Food Court,Flea Market,Fish & Chips Shop,Fast Food Restaurant,Ethiopian Restaurant,Electronics Store,Drugstore
9,5617.0,Strijp S,6,Stadsdeel Strijp,51.445473,5.455874,Eindhoven,1,Deli / Bodega,Ice Cream Shop,Office,Skate Park,Rock Club,Restaurant,Hotel Bar,Vegetarian / Vegan Restaurant,Coffee Shop,College Gym


# Mapping the clusters over Eindhoven

In [28]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(eind_label['Latitude'], eind_label['Longitude'], eind_label['Neighborhood'], eind_label['Label']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters